In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
import f90nml

rc('text', usetex=True)
rc('font', family='serif')
thin = 1
fs = 14
plt.rcParams["font.size"] = 12

figure_directory = "../figure/floquet_simulation/"
data_dir = '../data/floquet_simulation/'
if not os.path.isdir(figure_directory):
    os.makedirs(figure_directory)


def read_control(control_file):
    namelist = f90nml.read(data_dir + control_file)
    tmp = namelist['nml_parameter']
    N_time = tmp['N_time']
    NN = tmp['NN']
    NM = tmp['NM']
    NR = tmp['NR']
    NE = tmp['NE']
    N_min = tmp['N_min']
    N_max = tmp['N_max']
    M_min = tmp['M_min']
    M_max = tmp['M_max']
    Ro_min = tmp['Ro_min']
    Ro_max = tmp['Ro_max']
    e_min = tmp['e_min']
    e_max = tmp['e_max']
    return (N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
            Ro_min, Ro_max, e_min, e_max)


control_file = 'floquet_simulation.out'
(N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
 Ro_min, Ro_max,e_min, e_max) = read_control(control_file)

log_N_axis = np.linspace(np.log(N_min), np.log(N_max), NN, endpoint=True)
N_axis = np.exp(log_N_axis)
log_M_axis = np.linspace(np.log(M_min), np.log(M_max), NM, endpoint=True)
M_axis = np.exp(log_M_axis)
Ro_axis = np.linspace(Ro_min, Ro_max, NR, endpoint=True)
e_axis = np.linspace(e_min, e_max, NE, endpoint=True)

shape = np.array([4, N_time+1])
size = shape.prod()
data_type = np.dtype('<f8')
data_bytes = round(int(data_type.name[-2:]) / 8)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
fs = 16

def plot_series(IN, IR, IE, IM, ax):
    N = N_axis[IN-1]
    M = M_axis[IM-1]
    Ro = Ro_axis[IR-1]
    e = e_axis[IE-1]
    print(N, M, Ro, e)
    
    alpha_minus = Ro * (1 - e)**2 / (1 + (1 - e)**2)
    alpha_plus = Ro / (1 + (1 - e)**2)
    freq_vortex = np.sqrt(alpha_plus * alpha_minus)
    time_end = 2 * np.pi / freq_vortex
    time_axis = np.linspace(0, time_end, N_time+1, endpoint=True)
    
    data_file = (
        data_dir + 'unstable_series_{:0>6}_{:0>6}_{:0>6}_{:0>6}.out'.format(
            IN-1, IR-1, IE-1, IM-1
        ))
    file_open = open(data_file, 'r')
    data_tmp = np.fromfile(file_open, dtype=data_type, count=size)
    unstable_time_series = data_tmp.reshape(shape, order='F')
    U = (unstable_time_series[0, :] * np.cos(freq_vortex * time_axis)
         - unstable_time_series[1, :] * np.sin(freq_vortex * time_axis))
    V = (unstable_time_series[0, :] * np.sin(freq_vortex * time_axis)
         + unstable_time_series[1, :] * np.cos(freq_vortex * time_axis))
    W = unstable_time_series[2, :]
    T = unstable_time_series[3, :]
    
    U = np.sqrt(2.e0) * U
    V = np.sqrt(2.e0) * V
    W = np.sqrt(2.e0) * W
    T = np.sqrt(2.e0) * T
    
    print(
        np.log(U[-1] / U[0]) / time_axis[-1],
        np.log(V[-1] / V[0]) / time_axis[-1],
        np.log(W[-1] / W[0]) / time_axis[-1],
        np.log(T[-1] / T[0]) / time_axis[-1]
    )
    print((U[0]**2 + V[0]**2 + W[0]**2 + T[0]**2)/2)

    amplification = U[-1] / U[0]
    U_periodic = U[:] / amplification ** (time_axis / time_end)
    V_periodic = V[:] / amplification ** (time_axis / time_end)
    W_periodic = W[:] / amplification ** (time_axis / time_end)
    T_periodic = T[:] / amplification ** (time_axis / time_end)
    print((np.mean(U_periodic[:-1]**2 + V_periodic[:-1]**2 + W_periodic[:-1]**2
          + T_periodic[:-1]**2)) / 2)
          
    U_ext = np.concatenate([U, U[1:] * amplification])
    V_ext = np.concatenate([V, V[1:] * amplification])
    W_ext = np.concatenate([W, W[1:] * amplification])
    T_ext = np.concatenate([T, T[1:] * amplification])

    U_ext = np.concatenate([U_ext, U[1:] * amplification**2])
    V_ext = np.concatenate([V_ext, V[1:] * amplification**2])
    W_ext = np.concatenate([W_ext, W[1:] * amplification**2])
    T_ext = np.concatenate([T_ext, T[1:] * amplification**2])

    U_ext = np.concatenate([U_ext, U[1:] * amplification**3])
    V_ext = np.concatenate([V_ext, V[1:] * amplification**3])
    W_ext = np.concatenate([W_ext, W[1:] * amplification**3])
    T_ext = np.concatenate([T_ext, T[1:] * amplification**3])

    time_axis_ext = np.linspace(0, time_end*4, N_time*4+1, endpoint=True)
    time_axis_normal = time_axis_ext / time_end

    ax.plot(time_axis_normal, U_ext, c='r',
            label=r'$\hat{u}^r$', zorder=20);
    ax.plot(time_axis_normal, V_ext, c='b',
            label=r'$\hat{v}^r$', zorder=20);
    ax.plot(time_axis_normal, W_ext, c='m',
            label=r'$\hat{w}$', zorder=20);
    ax.plot(time_axis_normal, T_ext, c='g',
            label=r'$\hat{\theta}$', zorder=20);
    ax.hlines(xmin=0, xmax=4, y=0, color='grey', zorder=15)

    grid_x_ticks_minor = np.arange(0, 4.1, 0.1)
    grid_x_ticks_major = np.arange(0, 4.5, 0.5)
    grid_y_ticks_minor = np.arange(-35, 40, 5)
    grid_y_ticks_major = np.arange(-30, 40, 10)
    
    ax.set_xticks(grid_x_ticks_minor, minor=True)
    ax.set_xticks(grid_x_ticks_major, major=True)
    ax.set_yticks(grid_y_ticks_minor, minor=True)
    ax.set_yticks(grid_y_ticks_major, major=True)
    ax.set_xlim([0, 4])
    ax.set_ylim([-35, 35])
    ax.grid(zorder=10)
    ax.set_xlabel(r'$t/T$', fontsize=fs)
    
    U_freq = np.fft.rfft(U_periodic[:-1])
    V_freq = np.fft.rfft(V_periodic[:-1])
    W_freq = np.fft.rfft(W_periodic[:-1])
    T_freq = np.fft.rfft(T_periodic[:-1])
    
    KE_spec = np.zeros(int(N_time/2+1))
    KE_spec[0] = (np.abs(U_freq[0])**2 + np.abs(V_freq[0])**2
                  + np.abs(W_freq[0])**2)
    KE_spec[-1] = (np.abs(U_freq[-1])**2 + np.abs(V_freq[-1])**2
                  + np.abs(W_freq[-1])**2)
    KE_spec[1:-1] = (np.abs(U_freq[1:-1])**2 + np.abs(V_freq[1:-1])**2
                  + np.abs(W_freq[1:-1])**2) * 2
    KE_spec = KE_spec / N_time**2 / 2
    
    PE_spec = np.zeros(int(N_time/2+1))
    PE_spec[0] = np.abs(T_freq[0])**2
    PE_spec[-1] = np.abs(T_freq[-1])**2
    PE_spec[1:-1] = np.abs(T_freq[1:-1])**2 * 2
    PE_spec = PE_spec / N_time**2 / 2
    
    freq_axis = (np.linspace(0, N_time/2, int(N_time/2+1), endpoint=True)
                 * freq_vortex)
    print(np.sum(KE_spec + PE_spec))
    
    axins = inset_axes(ax, width=1.8, height=0.9, loc=3, borderpad=1.8)
    axins.bar(freq_axis / freq_vortex, KE_spec + PE_spec,
              align='center', width=0.3, color='navy');
    axins.set_xlim([0, 8]);
    axins.set_ylim([0, 2]);
    axins.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
    axins.xaxis.label.set_fontsize(12)
    axins.set_yticks([0, 1])
    axins.yaxis.label.set_fontsize(12)

IN = 9 + 1
IR = 15 + 1
IE = 12 + 1

fig = plt.figure(figsize=[6, 12])
fig.subplots_adjust(left=0.07, bottom=0.08, right=0.97,
                    top=0.95, wspace=0.2, hspace=0.25)
ax1 = fig.add_subplot(311)

plot_series(IN, IR, IE, 43937+1, ax1);
ax1.set_title('(a)', loc='left', fontsize=fs)
ax1.legend(fontsize=14, loc=2, ncol=2);

ax2 = fig.add_subplot(312)
plot_series(IN, IR, IE, 36154+1, ax2);
ax2.set_title('(b)', loc='left', fontsize=fs)
ax2.legend(fontsize=14, loc=2, ncol=2);

ax3 = fig.add_subplot(313)
plot_series(IN, IR, IE, 31680+1, ax3);
ax3.set_title('(c)', loc='left', fontsize=fs)
ax3.legend(fontsize=14, loc=2, ncol=2);

figure_name = figure_directory + '/' + 'floquet_unstable'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
fs = 16


IN = 9 + 1
IE = 16 + 1
IR = 18 + 1
IM = 29819 + 1

N = N_axis[IN-1]
M = M_axis[IM-1]
Ro = Ro_axis[IR-1]
e = e_axis[IE-1]
print(N, M, Ro, e)

alpha_minus = Ro * (1 - e)**2 / (1 + (1 - e)**2)
alpha_plus = Ro / (1 + (1 - e)**2)
freq_vortex = np.sqrt(alpha_plus * alpha_minus)
time_end = 2 * np.pi / freq_vortex
time_axis = np.linspace(0, time_end, N_time+1, endpoint=True)
time_axis_normal = time_axis / time_end

ax_ratio = np.sqrt(alpha_plus / alpha_minus)
K_series = np.cos(freq_vortex * time_axis)
L_series = - ax_ratio * np.sin(freq_vortex * time_axis)

data_file = (
    data_dir + 'neutral_series_{:0>6}_{:0>6}_{:0>6}_{:0>6}.out'.format(
        IN-1, IR-1, IE-1, IM-1
    ))
file_open = open(data_file, 'r')
data_tmp = np.fromfile(file_open, dtype=data_type, count=size)
unstable_time_series = data_tmp.reshape(shape, order='F')
U = (unstable_time_series[0, :] * np.cos(freq_vortex * time_axis)
     - unstable_time_series[1, :] * np.sin(freq_vortex * time_axis))
V = (unstable_time_series[0, :] * np.sin(freq_vortex * time_axis)
     + unstable_time_series[1, :] * np.cos(freq_vortex * time_axis))
W = unstable_time_series[2, :]
T = unstable_time_series[3, :]

U = np.sqrt(2.e0) * U
V = np.sqrt(2.e0) * V
W = np.sqrt(2.e0) * W
T = np.sqrt(2.e0) * T

U0 = np.sqrt(2.e0) * unstable_time_series[0, :]
V0 = np.sqrt(2.e0) * unstable_time_series[1, :]

Omega = 1.0 - alpha_plus - alpha_minus
denom = N**2 * K_series**2 + N**2 * L_series**2 + Omega**2 * M**2

Q = N * (K_series * V0 - L_series * U0) + Omega * M * T
D = K_series * U0 + L_series * V0 + M * W
U0V = - N * L_series / denom * Q
V0V = N * K_series / denom * Q
WV = np.zeros(N_time+1)
TV = Omega * M / denom * Q

U0W = U0 - U0V
V0W = V0 - V0V
WW = W - WV
TW = T - TV

UV = (U0V * np.cos(freq_vortex * time_axis)
      - V0V * np.sin(freq_vortex * time_axis))
VV = (U0V * np.sin(freq_vortex * time_axis)
      + V0V * np.cos(freq_vortex * time_axis))
UW = U - UV
VW = V - VV

EV = (UV[:]**2 + VV[:]**2 + WV[:]**2 + TV[:]**2) / 2
EW = (UW[:]**2 + VW[:]**2 + WW[:]**2 + TW[:]**2) / 2

fig = plt.figure(figsize=[6, 7])
fig.subplots_adjust(left=0.07, bottom=0.08, right=0.97,
                    top=0.95, wspace=0.2, hspace=0.25)

ax1 = fig.add_subplot(211)
ax1.plot(time_axis_normal, U, c='r',
         label=r'$\hat{u}^r$');
ax1.plot(time_axis_normal, V, c='b',
         label=r'$\hat{v}^r$');
ax1.plot(time_axis_normal, W, c='m',
         label=r'$\hat{w}$');
ax1.plot(time_axis_normal, T, c='g',
         label=r'$\hat{\theta}$');
ax1.hlines(xmin=0, xmax=4, y=0, color='grey')

grid_x_ticks_minor = np.arange(0, 1.01, 0.01)
grid_x_ticks_major = np.arange(0, 1.1, 0.1)
grid_y_ticks_minor = np.arange(-2.0, 2.1, 0.1)
grid_y_ticks_major = np.arange(-2.0, 2.5, 0.5)

ax1.set_xticks(grid_x_ticks_minor, minor=True)
ax1.set_xticks(grid_x_ticks_major, major=True)
ax1.set_yticks(grid_y_ticks_minor, minor=True)
ax1.set_yticks(grid_y_ticks_major, major=True)
ax1.set_xlim([0, 1])
ax1.set_ylim([-2, 2])
ax1.grid(zorder=10)
ax1.set_xlabel(r'$t/T$', fontsize=fs)
ax1.set_title('(a)', loc='left', fontsize=fs)
ax1.legend(fontsize=14, loc=1, ncol=2, framealpha=.9);

ax2 = fig.add_subplot(212)
ax2.plot(time_axis_normal, EV, c='r', label=r'$\hat{E}_v$');
ax2.plot(time_axis_normal, EW, c='b', label=r'$\hat{E}_w$');
ax2.plot(time_axis_normal, EV+EW, c='k', label=r'$\hat{E}$');

grid_x_ticks_minor = np.arange(0, 1.01, 0.01)
grid_x_ticks_major = np.arange(0, 1.1, 0.1)
grid_y_ticks_minor = np.arange(0, 2.1, 0.1)
grid_y_ticks_major = np.arange(0, 2.5, 0.5)

ax2.set_xticks(grid_x_ticks_minor, minor=True)
ax2.set_xticks(grid_x_ticks_major, major=True)
ax2.set_yticks(grid_y_ticks_minor, minor=True)
ax2.set_yticks(grid_y_ticks_major, major=True)
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 2])
ax2.grid(zorder=10)
ax2.set_xlabel(r'$t/T$', fontsize=fs)
ax2.set_title('(b)', loc='left', fontsize=fs)
ax2.legend(fontsize=14, loc=1, ncol=3, framealpha=.9);

figure_name = figure_directory + '/' +  'floquet_neutral'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')